In [1]:
conda install numpy

Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd 
import numpy as np 

from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.cluster import DBSCAN


import plotly.express as px
import plotly.io as pio
pio.renderers.default = "iframe_connected"

In [3]:
df_apr = pd.read_csv('uber-raw-data-apr14.csv')

In [4]:
df_may = pd.read_csv('uber-raw-data-may14.csv')

In [5]:
df_jun = pd.read_csv('uber-raw-data-jun14.csv')

In [6]:
df_jul = pd.read_csv('uber-raw-data-jul14.csv')

In [7]:
df_aug = pd.read_csv('uber-raw-data-aug14.csv')

In [8]:
df_sep = pd.read_csv('uber-raw-data-sep14.csv')

In [9]:
df_def = pd.concat([df_apr, df_aug, df_jul, df_jun, df_may, df_sep])

In [10]:
df_def.columns = [x.lower().replace('/', '_') for x in df_def.columns]

In [11]:
df_def.date_time = pd.to_datetime(df_def['date_time'])

In [12]:
new_dates, new_times = zip(*[(d.date(), d.time()) for d in df_def['date_time']])
df_def = df_def.assign(new_date=new_dates, new_time=new_times)
df_def['hour'] = df_def.date_time.dt.hour
df_def['day'] = df_def.date_time.dt.dayofweek

In [13]:
df_def = df_def.drop(columns=["date_time"])

In [14]:
len(df_def)

4534327

DROP OUTLIERS

In [15]:
for i in ['lat', 'lon',]:
    df_def = df_def[~(np.abs(df_def[i]-df_def[i].mean()) > (3*df_def[i].std()))]

In [16]:
len(df_def)

4301626

In [17]:
####################  TEST SUR DIFFERENTES HEURES ###########################################################

TEST : LUNDI A 18H

In [18]:
df_day_1_h18 = df_def[(df_def.day == 0) & (df_def.hour == 18)]


In [19]:
df_day_1_h18 = df_day_1_h18.drop(columns=['base', 'new_date', 'new_time'])

In [20]:
numeric_features = [0, 1, 2, 3] # Positions des colonnes quantitatives dans X
numeric_transformer = StandardScaler()

# On combine les transformers dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        
    ])

X_h18 = preprocessor.fit_transform(df_day_1_h18) # fit_transform !!

db = DBSCAN(eps=0.2, min_samples=100, algorithm="brute")

db.fit(X_h18)
np.unique(db.labels_)
df_day_1_h18["cluster"] = db.labels_


In [21]:
df_day_1_h18.head()

,lat,lon,hour,day,cluster
8638,40.7407,-73.9915,18,0,0
8639,40.7514,-73.9940,18,0,0
8640,40.7632,-73.9758,18,0,0
8641,40.7623,-73.9754,18,0,0
8642,40.7553,-73.9868,18,0,0


In [22]:
fig = px.scatter_mapbox(
        df_day_1_h18[(df_day_1_h18.cluster >= -1)], 
        lat="lat", 
        lon="lon",
        color="cluster",
        mapbox_style="carto-positron"
)

fig.show()

TEST : LUNDI A 1H DU MATIN

In [23]:
df_day_1_h1 = df_def[(df_def.day == 0) & (df_def.hour == 1)]

In [24]:
df_day_1_h1 = df_day_1_h1.drop(columns=['base', 'new_date', 'new_time'])

In [25]:
X_h1 = preprocessor.fit_transform(df_day_1_h1) # fit_transform !!

db = DBSCAN(eps=0.2, min_samples=100, algorithm="brute")

db.fit(X_h1)
np.unique(db.labels_)
df_day_1_h1["cluster"] = db.labels_


In [26]:
fig = px.scatter_mapbox(
        df_day_1_h1[(df_day_1_h1.cluster >= -1)], 
        lat="lat", 
        lon="lon",
        color="cluster",
        mapbox_style="carto-positron"
)

fig.show()

In [27]:
##############################################################################################################################

VISUALISATION DES LUNDI TOUTES LES HEURES.

In [28]:
df_def.head()

,lat,lon,base,new_date,new_time,hour,day
0,40.7690,-73.9549,B02512,2014-04-01,00:11:00,0,1
1,40.7267,-74.0345,B02512,2014-04-01,00:17:00,0,1
2,40.7316,-73.9873,B02512,2014-04-01,00:21:00,0,1
3,40.7588,-73.9776,B02512,2014-04-01,00:28:00,0,1
4,40.7594,-73.9722,B02512,2014-04-01,00:33:00,0,1


In [29]:
df_j1 = df_def[(df_def.day == 0) & (df_def.hour <=23)]

In [30]:
df_j1

,lat,lon,base,new_date,new_time,hour,day
7785,40.7205,-73.9939,B02512,2014-04-07,00:31:00,0,0
7786,40.7407,-74.0077,B02512,2014-04-07,00:37:00,0,0
7787,40.7591,-73.9892,B02512,2014-04-07,00:50:00,0,0
7788,40.7419,-74.0034,B02512,2014-04-07,00:58:00,0,0
7789,40.7419,-74.0034,B02512,2014-04-07,01:27:00,1,0
...,...,...,...,...,...,...,...
1020560,40.7028,-73.9088,B02764,2014-09-29,23:55:00,23,0
1020561,40.6920,-73.9912,B02764,2014-09-29,23:55:00,23,0
1020562,40.7323,-73.9879,B02764,2014-09-29,23:56:00,23,0
1020564,40.7193,-74.0474,B02764,2014-09-29,23:57:00,23,0


In [31]:
df_j1 = df_j1.drop(columns=['base', 'new_date', 'new_time'])

In [32]:
df_j1

,lat,lon,hour,day
7785,40.7205,-73.9939,0,0
7786,40.7407,-74.0077,0,0
7787,40.7591,-73.9892,0,0
7788,40.7419,-74.0034,0,0
7789,40.7419,-74.0034,1,0
...,...,...,...,...
1020560,40.7028,-73.9088,23,0
1020561,40.6920,-73.9912,23,0
1020562,40.7323,-73.9879,23,0
1020564,40.7193,-74.0474,23,0


In [33]:
numeric_features = [0, 1, 2, 3] # Positions des colonnes quantitatives dans X
numeric_transformer = StandardScaler()

# On combine les transformers dans un ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        
    ])

X_total = preprocessor.fit_transform(df_j1) # fit_transform !!

db = DBSCAN(eps=0.2, min_samples=100, algorithm="brute")

db.fit(X_total)
np.unique(db.labels_)
df_j1["cluster"] = db.labels_


In [34]:
fig = px.scatter_mapbox(df_j1[(df_j1.cluster >=-1)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="cluster",
        mapbox_style="carto-positron",
        animation_frame="hour"
)
fig.show()

VISUALITION MARDI TOUTES LES HEURES

In [35]:
df_j1 = df_def[(df_def.day == 0) & (df_def.hour <=23)].drop(columns=['base', 'new_date', 'new_time'])

In [38]:
X_total = preprocessor.fit_transform(df_j1) # fit_transform !!

db = DBSCAN(eps=0.2, min_samples=100, algorithm="brute")

db.fit(X_total)
np.unique(db.labels_)
df_j1["cluster"] = db.labels_

In [39]:
fig = px.scatter_mapbox(df_j1[(df_j1.cluster >=-1)], 
        lat="lat", 
        lon="lon",
        hover_name = 'hour',                
        color="cluster",
        mapbox_style="carto-positron",
        animation_frame="hour"
)
fig.show()